In [1]:
import meteo_utils
from datetime import date, datetime, timedelta
import pandas as pd
import plotly.express as px
import numpy as np

## Парсинг данных с сайта

In [96]:
station_id = 28745      # идентификатор станции, с которой нужны наблюдения    
start = date(2022,1,1) # начало периода загрузки данных 
end = date(2024,12,31)      # конец  периода загрузки данных  

meteo_utils.load_weaclim4period (station_id, start, end, 'weaclim_data')

Loading weaclim data for 28745: 100%|██████████| 36/36 [02:00<00:00,  3.34s/it, Processing: 2024-12]


## Чтение скачанных и сохраненных данных

In [2]:
path_ch = r'weaclim_data\28645'
path_uzh = r'weaclim_data\28745'
df_ch = meteo_utils.read_weaclim_dir(path_ch) # Челябинск
df_u = meteo_utils.read_weaclim_dir(path_uzh) # Южноуральск

df_ch = df_ch.rename(columns = {x:'ch_' + x for x in df_ch.columns})
df_u = df_u.rename(columns = {x:'u_' + x for x in df_u.columns})

df = df_ch.merge (df_u, left_index=True, right_index=True)
df.head(10)

,ch_t2m,ch_rh2m,ch_slp,ch_ps,ch_vel10m,ch_gust10m,ch_tcc,ch_lcc,ch_dir10m,u_t2m,u_rh2m,u_slp,u_ps,u_vel10m,u_gust10m,u_tcc,u_lcc,u_dir10m
Datetime,,,,,,,,,,,,,,,,,,
2022-01-01 12:00:00,-9.9,87.0,1012.8,982.6,2.0,NaN,10.0,10.0,180.0,-10.3,87,1013.9,982.9,2.0,NaN,10.0,8.0,180.0
2022-01-01 15:00:00,-10.0,87.0,1012.1,981.9,1.0,NaN,10.0,10.0,180.0,-9.0,85,1012.8,982.0,3.0,NaN,10.0,0.0,180.0
2022-01-01 18:00:00,-9.2,85.0,1011.0,980.9,0.0,NaN,10.0,10.0,NaN,-9.1,83,1012.1,981.3,2.0,NaN,10.0,0.0,180.0
2022-01-01 21:00:00,-9.6,88.0,1010.5,980.4,1.0,NaN,10.0,10.0,180.0,-9.3,84,1011.3,980.5,1.0,NaN,10.0,0.0,180.0
2022-01-02 00:00:00,-11.0,95.0,1009.6,979.3,2.0,NaN,10.0,0.0,180.0,-8.0,88,1010.2,979.6,2.0,NaN,10.0,0.0,180.0
2022-01-02 03:00:00,-9.8,91.0,1008.3,978.2,2.0,NaN,10.0,0.0,180.0,-8.7,89,1009.5,978.8,2.0,NaN,10.0,0.0,180.0
2022-01-02 06:00:00,-5.1,91.0,1006.9,977.4,2.0,NaN,10.0,10.0,180.0,-6.1,90,1008.2,977.9,2.0,NaN,10.0,10.0,180.0
2022-01-02 09:00:00,-3.7,93.0,1005.3,976.1,2.0,NaN,10.0,10.0,180.0,-4.5,89,1006.3,976.2,1.0,NaN,10.0,10.0,180.0
2022-01-02 12:00:00,-4.1,94.0,1003.8,974.5,1.0,NaN,10.0,10.0,180.0,-6.9,89,1005.3,974.9,2.0,NaN,10.0,10.0,180.0


In [3]:
path_ch = r'weaclim_data\28645'
df = meteo_utils.read_weaclim_dir(path_ch)
df = df.iloc[4280:4400, :]

In [4]:
df['hour'] = df.index.hour
df['date'] = df.index.strftime('%d.%m.%Y')
df['lon'] = 61.29
df['lat'] = 55.15
df['clouds'] = df['tcc'] * 8 / 10
df['sealvl'] = 220
raym = df[['date', 'hour', 'lon', 'lat', 'sealvl','t2m', 'rh2m', 'vel10m', 'clouds', 't2m']]
raym.index = raym['date']
raym.drop('date', axis=1, inplace=True)
raym.to_csv('chelyabinsk.txt')
raym

C:\Users\stasy\AppData\Local\Temp\ipykernel_15636\3239195283.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raym.drop('date', axis=1, inplace=True)


,hour,lon,lat,sealvl,t2m,rh2m,vel10m,clouds,t2m
date,,,,,,,,,
01.07.2023,21,61.29,55.15,220,19.6,60.0,1.0,3.2,19.6
02.07.2023,0,61.29,55.15,220,19.9,55.0,2.0,4.0,19.9
02.07.2023,3,61.29,55.15,220,23.2,41.0,3.0,6.4,23.2
02.07.2023,6,61.29,55.15,220,26.9,32.0,3.0,4.8,26.9
02.07.2023,9,61.29,55.15,220,25.9,38.0,1.0,8.0,25.9
...,...,...,...,...,...,...,...,...,...
16.07.2023,6,61.29,55.15,220,17.0,76.0,0.0,8.0,17.0
16.07.2023,9,61.29,55.15,220,21.1,61.0,1.0,6.4,21.1
16.07.2023,12,61.29,55.15,220,21.2,55.0,2.0,8.0,21.2


In [5]:
fig = px.line(df_monmean, y=['u_t2m', 'ch_t2m'], title='Simple Time Series Plot') #, labels={'Value': 'Cumulative Value'})
fig.data[0].line.color = 'blue'  # Color for 'rur_t2m'
fig.data[1].line.color = 'orange'  # Color for 'urb_t2m'
fig.show()

NameError: name 'df_monmean' is not defined

<h2>Построение графиков осредненного сезонного хода метеовеличин и их разностей</h2>

In [8]:
for mon, mon_name in zip([1, 7], ['Январе', 'Июле']):
    df_mon = df[df.index.month.isin([mon])]
    daily_mon_mean = df_mon.groupby(df_mon.index.day).mean()
    for param, name in zip(['t2m', 'rh2m', 'vel10m'], ['температур на 2 метрах', 'влажности на 2 метрах', 'скорости ветра на 10 метрах']):
        daily_mon_mean[f'dif_{param}'] = daily_mon_mean[f'ch_{param}'] - daily_mon_mean[f'u_{param}']
        fig = px.line(daily_mon_mean, y=[f'u_{param}', f'ch_{param}', f'dif_{param}'], title=f'Значения {name} в {mon_name}', width=700, height=400) #, labels={'Value': 'Cumulative Value'})
        fig.data[0].line.color = 'blue'  # Color for 'rur_t2m'
        fig.data[1].line.color = 'orange'  # Color for 'urb_t2m'
        fig.data[2].line.color = 'gray'  # Color for 'rur_t2m'
        fig.update_layout()
        fig.show()

<h4> Температура на 2 метрах</h4>Южноуральск (u_t2m) и Челябинск (ch_t2m) демонстрируют схожую динамику изменения температуры как в январе, так и в июле.Разница температур (dif_t2m) между станциями минимальна, что указывает на схожие климатические условия, хотя в зимний период разница может быть немного выраженнее.

<h4>Влажность на 2 метрах</h4>Уровни влажности (u_rh2m и ch_rh2m) показывают заметную схожесть, особенно в летний период. В зимний период наблюдаются различия, особенно в пиковых значениях. Разница влажности (dif_rh2m) остается в пределах ±10% в большинстве случаев, что говорит о незначительном влиянии городской среды на влажность.

<h4>Скорость ветра на 10 метрах</h4>Данные скорости ветра (u_vel10m и ch_vel10m) имеют более заметные различия, особенно в зимний период. Это может быть связано с различиями в рельефе, застройке и локальных метеорологических условиях. Разница в скорости ветра (dif_vel10m) иногда достигает значений около 1 м/с, что может быть обусловлено урбанистическими особенностями Челябинска.

<h4>Сравнение январь/июль</h4>Летние месяцы характеризуются меньшими различиями в температуре и влажности между станциями, что свидетельствует о более однородных климатических условиях. Зимой, напротив, разница становится заметнее, особенно в скорости ветра и влажности, что может быть связано с влиянием эффекта теплового острова на микроклимат.

<h4>Выводы</h4>Южноуральск демонстрирует типичный фон, тогда как Челябинск показывает влияние городской среды, особенно зимой (различия в скорости ветра и влажности). Влияние урбанистических факторов на температуру менее выражено, но зимой может быть заметно в минимальных температурах.

<h2>Построение графиков осредненного суточного хода метеовеличин и их разностей</h2>

In [3]:
df['hour'] = df.index.hour
daily_cycle = df.groupby('hour').mean()
for param, name in zip(['t2m', 'rh2m', 'vel10m'], ['температур на 2 метрах', 'влажности на 2 метрах', 'скорости ветра на 10 метрах']):
    daily_cycle[f'dif_{param}'] = daily_cycle[f'ch_{param}'] - daily_cycle[f'u_{param}']
    fig = px.line(daily_cycle, y=[f'u_{param}', f'ch_{param}', f'dif_{param}'], title=f'Среднесуточные значения {name}', width=700, height=400) #, labels={'Value': 'Cumulative Value'})
    fig.data[0].line.color = 'blue'  # Color for 'rur_t2m'
    fig.data[1].line.color = 'orange'  # Color for 'urb_t2m'
    fig.data[2].line.color = 'gray'  # Color for 'rur_t2m'
    fig.update_layout()
    fig.show()

<h4>Среднесуточные значения температуры на 2 метрах</h4>
Температура на фоновой (u_t2m) и городской (ch_t2m) станциях имеет схожий суточный ход.
Максимальные значения наблюдаются около 14:00–15:00, минимальные — в ранние утренние часы.
Разница температуры (dif_t2m), остается минимальной (менее 0,5 °C) в течение суток, что подтверждает отсутствие выраженного влияния урбанизации на температуру.

<h4>Среднесуточные значения влажности на 2 метрах</h4>
Влажность на обоих станциях демонстрирует аналогичный ход: максимальные значения приходятся на ночное время, минимальные — на дневное.
Разница влажности (dif_rh2m) также невелика, не превышает 3–4%. Это указывает на схожие условия испарения и уровень влагообмена между станциями.
В дневное время влажность немного ниже на городской станции, вероятно, из-за меньшего количества растительности и водных поверхностей.

<h4>Среднесуточные значения скорости ветра на 10 метрах</h4>

Скорость ветра на обеих станциях имеет дневной пик около 12:00–14:00 и снижение в вечерние часы.
Городская станция (ch_vel10m) показывает чуть более высокие скорости ветра в дневное время, что может быть связано с усилением турбулентности из-за городской застройки.
Разница скорости ветра (dif_vel10m) колеблется в пределах ±0,5 м/с, что указывает на умеренное влияние урбанизации на динамику ветра.

* Рассчитать для города и фона градусо-дни отопительного и охладительного периодов (heating/cooling degree days, HDD/CDD). В качестве критерия расчета таких дней можно использовать нормы для Европейского союза с пороговыми значениями среднесуточной температуры 15 и 24 °C, или обосновать использование других пороговых значений для выбранного города.

<h2>Расчет градусо-дней отопительного и охладительного периодов</h2>

In [4]:
daily_data = df.groupby(df.index.date)['ch_t2m'].mean().reset_index()
daily_data['u_t2m'] = df.groupby(df.index.date)['u_t2m'].mean().reset_index()['u_t2m']

def calculate_degree_days(avg_temp):
    if avg_temp < 15:
        return (15 - avg_temp, 0)  # HDD, CDD
    elif avg_temp > 24:
        return (0, avg_temp - 24)  # HDD, CDD
    else:
        return (0, 0)

daily_data[['ch_HDD', 'ch_CDD']] = daily_data['ch_t2m'].apply(calculate_degree_days).apply(pd.Series)
daily_data[['u_HDD', 'u_CDD']] = daily_data['u_t2m'].apply(calculate_degree_days).apply(pd.Series)

print(f'Сумма градусо-дней в Челябинске: отопительный период - {daily_data['ch_HDD'].sum().round(2)}, охладительный период - {daily_data['ch_CDD'].sum().round(2)}')
print(f'Сумма градусо-дней в Южноуральске: отопительный период - {daily_data['u_HDD'].sum().round(2)}, охладительный период - {daily_data['u_CDD'].sum().round(2)}')

Сумма градусо-дней в Челябинске: отопительный период - 13127.6, охладительный период - 51.49
Сумма градусо-дней в Южноуральске: отопительный период - 12861.51, охладительный период - 81.05


Сумма градусо-дней в Челябинске: отопительный период - 13127.6, охладительный период - 51.49<br>Сумма градусо-дней в Южноуральске: отопительный период - 12861.51, охладительный период - 81.05